# 大数据之路
## 数据技术
## 数据模型
    - 典型的数仓建设方法论
        - ER模型
        - 维度模型
        - Data Vault模型
        - Anchor模型
    - 演变
        - ODS+DSS -> ODL+BDL+IDL+ADL -> OneData
### 名词术语
- 数据域
- 业务过程(事件)
- 度量/原子指标: 如支付金额
- 时间周期
- 修饰类型: 如终端类型
- 修饰词: 如手机支付、卡支付
- 维度(实体对象): 如地域维度、时间维度
- 维度属性: 如地域维度中的国家、省、市、区等
- 派生指标: 原子指标+多个修饰词(可选)+时间周期, 如近一年的车票支付金额

### 模型层次
1. ODS
1. CDM
    - DWD
    - DWS
1. ADS

## 数据管理
## 数据应用

# SparkSQL内核剖析
## Spark背景
## Spark基础知识
- RDD
- 窄依赖、宽依赖
- Shuffle
- 算子: transform、action
- DAG、checkpoint
- Driver程序

## Spark SQL执行全过程概述

### 示例

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [5]:
import os

# df = spark.read.json(f"{os.environ['SPARK_HOME']}/examples/src/main/resources/people.json")
df = spark.read.json('data/student.json')
df.show()

df.createOrReplaceTempView('student')

spark.sql('select name from student where age > 18').show()

+---+---+----+
|age| id|name|
+---+---+----+
| 29|  1|Kate|
| 30|  2|Andy|
| 10|  3|Tony|
+---+---+----+

+----+
|name|
+----+
|Kate|
|Andy|
+----+



### 逻辑计划 & 物理计划
- LogicalPlan
    - Unresolved LogicalPlan
    - Analyzed LogicalPlan
    - Optimized LogicalPlan
- PhysicalPlan
    - Iterator[PhyysicalPlan]
    - SparkPlan
    - Prepared SparkPlan
- 从SQL语句的解析一直到提交之前,整个转换过程都在Spark集群的Driver端进行,不涉及分布式环境

### 环境准备
1. 收集sparkPlan的jar准备
    - 创建demo-scala工程: 实现`org.apache.spark.sql.util.QueryExecutionListener`
    - 收集sparkPlan写入`spark-plan-log/${funcName}_${uuid}.json`文件
    - demo-scala package => demo-scala-1.0-SNAPSHOT.jar,放入samples/目录下(与spark-plan-log同级)
1. 编写ss.cmd放入`samples/`
    ```cmd
    @echo on

    rem
    rem 指定queryExecutionListeners参数的spark-submit.cmd脚本
    rem

    cmd /V /E /C "spark-submit.cmd  --conf spark.sql.queryExecutionListeners=org.example.framework.LineageListener --jars demo-scala-1.0-SNAPSHOT.jar %*"
    ```
1. 创建`samples/sparksqlcore/`目录: 存放示例代码
1. 创建`samples/spark-plan-log/`目录: 存放sparkPlan json数据
1. 编写`samples/sparksqlcore/demo3_1.py`:
    ```python
    from pyspark.sql import SparkSession
    import os

    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .getOrCreate()

    df = spark.read.json('data/student.json')
    df.show()

    df.createOrReplaceTempView('student')

    spark.sql('select name from student where age > 18').show()
    ```
1. 切换目录至`samples/`下,执行`ss sparksqlcore/demo3_1.py`
1. 查看`spark-plan-log/`收集的执行计划

### 分析
- `select name from student where age > 18`
- sparkPlan
    - 对应数据表: org.apache.spark.sql.execution.FileSourceScanExec
    - 过滤逻辑:   org.apache.spark.sql.execution.FilterExec
    - 列裁剪逻辑: org.apache.spark.sql.execution.ProjectExec
    - 应该是自动添加的: org.apache.spark.sql.execution.CollectLimitExec

### Catalyst
- InternalRow
- TreeNode
- Expression

## Spark SQL编译器Parser
- DSL(Domain Specific Language)
- 非图灵完备的语言
- ANTLR : another tool for language recognition
    - ANTLR4
    - sql/api/src/main/antlr4/org/apache/spark/sql/catalyst/parser/SqlBaseParser.g4(https://github.com/apache/spark/blob/master/sql/api/src/main/antlr4/org/apache/spark/sql/catalyst/parser/SqlBaseParser.g4)
- 监听器模式
- 访问者模式